In [189]:
!pip install pycaret[full]

zsh:1: no matches found: pycaret[full]


In [54]:
import numpy as np
import pandas as pd

from pycaret.regression import *
from pycaret.utils import *
from sklearn.metrics import mean_squared_error
import datetime

In [206]:
dataset = pd.read_csv('datasets/big-table-train.csv')
dataset = dataset.dropna()
dataset = dataset.drop(columns=['completed_time', 'node_start', 'node_finish', 'lon_start', 'lon_end', 'lat_end', 'lat_start', 'average_speed'])

dataset

,Id,running_time,route_distance_km,delta_time,distance,speed
0,-4773019581999572651,2022-01-24 18:30:21,3.740,862.0,31.771489,30.0
3,-4773019581999572651,2022-01-24 18:30:21,3.740,862.0,50.034390,29.0
4,-4773019581999572651,2022-01-24 18:30:21,3.740,862.0,13.453126,30.0
5,-4773019581999572651,2022-01-24 18:30:21,3.740,862.0,15.994386,32.0
7,-4773019581999572651,2022-01-24 18:30:21,3.740,862.0,47.897295,30.0
...,...,...,...,...,...,...
401294,-4845695090750648713,2022-01-24 14:57:46,2.323,819.0,11.114741,18.0
401295,-4845695090750648713,2022-01-24 14:57:46,2.323,819.0,13.232601,21.0
401296,-4845695090750648713,2022-01-24 14:57:46,2.323,819.0,41.040972,17.0
401297,-4845695090750648713,2022-01-24 14:57:46,2.323,819.0,20.488592,19.0


In [207]:
dataset_agg = dataset.groupby(['Id', 'running_time', 'route_distance_km', 'delta_time'])\
    .agg({'distance': ['min', 'max', 'mean', 'std'], 'speed': ['min', 'max', 'mean', 'std']}).reset_index()

dataset_agg = dataset_agg.drop(columns=['Id'])
dataset_agg['running_time'] = (pd.to_datetime(dataset_agg['running_time']) - datetime.datetime(1970,1,1)).dt.total_seconds()
dataset_agg

running_time route_distance_km delta_time  distance              \
                                                      min         max   
0     1.643067e+09             3.179      469.0  2.494184  127.702093   
1     1.643058e+09             6.137      688.0  2.664808  176.030833   
2     1.643064e+09             6.312      683.0  1.261206  201.267382   
3     1.643063e+09             6.379      885.0  2.757002  231.663338   
4     1.643045e+09             2.551      612.0  1.735633  224.340608   
...            ...               ...        ...       ...         ...   
4990  1.643065e+09             5.520      753.0  1.306968  327.558417   
4991  1.643057e+09             3.146      462.0  1.688882  156.095828   
4992  1.643002e+09             4.038      607.0  4.213290  184.102066   
4993  1.643056e+09             1.641      223.0  1.306756   89.077514   
4994  1.643050e+09             2.489      368.0  8.476520  184.070599   

                           speed                              
           mean        std   min   max       mean        std  
0     33.761364  28.634283  11.0  43.0  25.342857   7.550575  
1     47.742523  42.632365  10.0  50.0  29.904110   8.487187  
2     33.584700  33.855656   7.0  47.0  33.028926   8.396502  
3     37.848757  38.958754  15.0  51.0  36.278846   9.887186  
4     31.892602  43.984874   9.0  83.0  24.270270  12.985025  
...         ...        ...   ...   ...        ...        ...  
4990  32.873786  46.691882  11.0  58.0  33.803704   9.062819  
4991  28.981445  30.362658  18.0  41.0  30.740964   6.662308  
4992  49.033223  41.811231  19.0  68.0  40.695652  10.822504  
4993  25.529379  20.015225  13.0  32.0  24.372093   4.556645  
4994  49.910280  46.718347  18.0  40.0  30.212121   5.128094  

[4995 rows x 11 columns]

In [208]:
dataset_agg = pd.DataFrame(dataset_agg.values, columns=['running_time', 'route_distance_km', 'delta_time', 'min_distance', 'max_distance', 'mean_distance', 'std_distance', 'min_speed', 'max_speed', 'mean_speed', 'std_speed'])
dataset_agg['delta_time'] = np.log1p(dataset_agg['delta_time'])
dataset_agg

,running_time,route_distance_km,delta_time,min_distance,max_distance,mean_distance,std_distance,min_speed,max_speed,mean_speed,std_speed
0,1.643067e+09,3.179,6.152733,2.494184,127.702093,33.761364,28.634283,11.0,43.0,25.342857,7.550575
1,1.643058e+09,6.137,6.535241,2.664808,176.030833,47.742523,42.632365,10.0,50.0,29.904110,8.487187
2,1.643064e+09,6.312,6.527958,1.261206,201.267382,33.584700,33.855656,7.0,47.0,33.028926,8.396502
3,1.643063e+09,6.379,6.786717,2.757002,231.663338,37.848757,38.958754,15.0,51.0,36.278846,9.887186
4,1.643045e+09,2.551,6.418365,1.735633,224.340608,31.892602,43.984874,9.0,83.0,24.270270,12.985025
...,...,...,...,...,...,...,...,...,...,...,...
4990,1.643065e+09,5.520,6.625392,1.306968,327.558417,32.873786,46.691882,11.0,58.0,33.803704,9.062819
4991,1.643057e+09,3.146,6.137727,1.688882,156.095828,28.981445,30.362658,18.0,41.0,30.740964,6.662308
4992,1.643002e+09,4.038,6.410175,4.213290,184.102066,49.033223,41.811231,19.0,68.0,40.695652,10.822504
4993,1.643056e+09,1.641,5.411646,1.306756,89.077514,25.529379,20.015225,13.0,32.0,24.372093,4.556645


In [209]:
data = dataset_agg.sample(frac=0.9, random_state=786)
data_unseen = dataset_agg.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (4496, 11)
Unseen Data For Predictions: (499, 11)


In [215]:
s = setup(data = data, target = 'delta_time', session_id=123, use_gpu=False, normalize = True)

,Description,Value
0,Session id,123
1,Target,delta_time
2,Target type,Regression
3,Original data shape,"(4496, 11)"
4,Transformed data shape,"(4496, 11)"
5,Transformed train set shape,"(3147, 11)"
6,Transformed test set shape,"(1349, 11)"
7,Numeric features,10
8,Rows with missing values,0.0%
9,Preprocess,True


In [216]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [217]:
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.1787,0.0605,0.2455,0.6217,0.0348,0.0289,0.0730
lightgbm,Light Gradient Boosting Machine,0.1821,0.0610,0.2468,0.6191,0.0349,0.0294,0.0170
et,Extra Trees Regressor,0.1824,0.0614,0.2475,0.6164,0.0350,0.0295,0.0820
rf,Random Forest Regressor,0.1816,0.0620,0.2487,0.6122,0.0352,0.0294,0.1610
xgboost,Extreme Gradient Boosting,0.1929,0.0705,0.2650,0.5589,0.0375,0.0312,0.0440
ridge,Ridge Regression,0.2101,0.0781,0.2791,0.5140,0.0392,0.0339,0.0070
lar,Least Angle Regression,0.2101,0.0781,0.2791,0.5140,0.0392,0.0339,0.0070
br,Bayesian Ridge,0.2102,0.0781,0.2791,0.5140,0.0392,0.0339,0.0060
lr,Linear Regression,0.2101,0.0781,0.2791,0.5140,0.0392,0.0339,0.0070
huber,Huber Regressor,0.2085,0.0787,0.2801,0.5104,0.0393,0.0337,0.0080


In [218]:
best_model = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1795,0.0585,0.2419,0.6732,0.0343,0.0291
1,0.1835,0.0663,0.2575,0.5283,0.0361,0.0294
2,0.1825,0.0666,0.2581,0.5628,0.0363,0.0294
3,0.1729,0.0598,0.2445,0.6225,0.0344,0.0279
4,0.1766,0.0580,0.2407,0.6662,0.0342,0.0287
5,0.1794,0.0558,0.2363,0.7084,0.0334,0.0291
6,0.1860,0.0588,0.2424,0.6352,0.0345,0.0301
7,0.1710,0.0565,0.2377,0.6396,0.0335,0.0277
8,0.1620,0.0476,0.2181,0.6682,0.0307,0.0261


In [ ]:
tuned_best_model = tune_model(best_model, n_iter=100)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:31:13
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [169]:
plot_model(tuned_best_model, 'feature')

TypeError: Feature Importance and RFE plots not available for estimators that doesnt support coef_ or feature_importances_ attribute.

In [170]:
interpret_model(tuned_best_model)

TypeError: This function only supports tree based models for binary classification: dt, xgboost, et, rf, lightgbm.

In [ ]:
# validate on unseen data
unseen_predictions = predict_model(tuned_best_model, data=data_unseen)
unseen_predictions

In [ ]:
(mean_squared_error(np.expm1(unseen_predictions['delta_time']), np.expm1(unseen_predictions['prediction_label']), squared=False))

### Model predictions on test data

In [42]:
big_table_test_df = pd.read_csv("datasets/big-table-test.csv")

In [43]:
submission_df = pd.read_csv("datasets/submission.csv")
submission_df

,Id,Predicted
0,6198,661.215529
1,6417,677.967213
2,7054,676.752201
3,9628,766.346661
4,10283,679.669995
...,...,...
995,525706,655.662647
996,526604,688.685152
997,527213,617.559343
998,527520,623.181735


In [44]:
submission_predictions = predict_model(tuned_best_model, data=big_table_test_df)
submission_predictions

,Id,running_time,route_distance_km,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end,is_work_hour,is_night_time,is_late_night_time,is_rush_hour,prediction_label
0,6198,2022-01-24 03:38:30,4.744,8952394129,8952394128,138.795715,37.0,47.573032,NaN,NaN,NaN,NaN,0,0,1,0,667.327283
1,6198,2022-01-24 03:38:30,4.744,2059503754,4548172320,95.273003,33.0,47.573032,30.724094,46.431126,30.724951,46.431747,0,0,1,0,693.168347
2,6198,2022-01-24 03:38:30,4.744,2059504508,2059503754,137.647888,35.0,47.573032,30.722883,46.430222,30.724094,46.431126,0,0,1,0,693.326201
3,6198,2022-01-24 03:38:30,4.744,1570776534,1977018578,4.383708,30.0,47.573032,NaN,NaN,NaN,NaN,0,0,1,0,663.109536
4,6198,2022-01-24 03:38:30,4.744,1977018580,1977018576,24.195593,31.0,47.573032,30.725685,46.437679,30.725998,46.437653,0,0,1,0,690.881037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102537,527850,2022-01-24 18:45:20,10.127,290891780,5957304897,40.933262,45.0,48.289474,30.735411,46.470631,30.735943,46.470589,1,0,0,1,675.616776
102538,527850,2022-01-24 18:45:20,10.127,5957304897,3902949792,63.421597,34.0,48.289474,30.735943,46.470589,30.736767,46.470528,1,0,0,1,689.397664
102539,527850,2022-01-24 18:45:20,10.127,3902949792,290404192,4.303810,16.0,48.289474,30.736767,46.470528,30.736851,46.470520,1,0,0,1,689.754772
102540,527850,2022-01-24 18:45:20,10.127,290404192,5957304888,5.250640,37.0,48.289474,30.736851,46.470520,30.736845,46.470474,1,0,0,1,691.851122


In [45]:
predicted_df = submission_predictions[['Id', "prediction_label"]].drop_duplicates(subset=["Id"], keep='first')
predicted_df

,Id,prediction_label
0,6198,667.327283
89,6417,678.709113
261,7054,679.337341
332,9628,720.408084
511,10283,679.016988
...,...,...
102016,525706,652.933221
102076,526604,660.790962
102167,527213,659.428835
102292,527520,512.766881


In [46]:
submission_df = submission_df \
    .merge(predicted_df, on='Id', how='inner') \
    .drop_duplicates(subset=["Id"], keep='first')
submission_df

,Id,Predicted,prediction_label
0,6198,661.215529,667.327283
1,6417,677.967213,678.709113
2,7054,676.752201,679.337341
3,9628,766.346661,720.408084
4,10283,679.669995,679.016988
...,...,...,...
995,525706,655.662647,652.933221
996,526604,688.685152,660.790962
997,527213,617.559343,659.428835
998,527520,623.181735,512.766881


In [47]:
submission_df["Predicted"] = submission_df["prediction_label"]
submission_df.drop("prediction_label", axis=1, inplace=True)

In [48]:
submission_df.to_csv("datasets/submission.csv", index=False)

## Submission phase

Guide: https://www.kaggle.com/code/derrickmwiti/how-to-make-submissions-using-kaggle-s-api/notebook

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ["KAGGLE_USERNAME"]="beardimon"
os.environ["KAGGLE_KEY"]="22adc3776f244a54f4732bf8dfb077eb"

In [ ]:
!kaggle competitions submit -c int20h-2023-hackathon  -f ./datasets/submission.csv -m "Baseline"

In [ ]:
!kaggle competitions leaderboard -c int20h-2023-hackathon --show

In [ ]:
!kaggle competitions submissions -c int20h-2023-hackathon